# AI - Vacuum agent, empty program

Created (R. Basili, Nov. 2020)  
Modified (C.D. Hromei, Oct 2024)

## Exercises

1. Fill the program of the `Agent` in order to reproduce the behaviour of random moves.
2. Fill the program of the `Agent` in order to search for the closest dirty room, go there and `Suck`.  
    **Hint**: you should modify the `Perception` method of the Environment as well to make it Completely Observable.
3. Modify the `Environment` locations into a 2x2 matrix instead of a row.
4. Modify the `Execute action` method of the Environment in order to implement a probabilistic failure of the action chosen by the Agent. In that case, the action should fail and nothing happens!

In [16]:
from tkinter import *
import random
from agents import *

collections.Callable = collections.abc.Callable

"""
loc_A loc_B

loc_C loc_D

"""
locs = ((0, 0), (1, 0), (0, 1), (1, 1))
pos = ((100, 100), (400, 100), (100, 300), (400, 300))
loc_A, loc_B, loc_C, loc_D = locs  # The three locations for the Vacuum world
pos_A, pos_B, pos_C, pos_D = pos  # The three locations for the Vacuum world

NBUTTONS = 4

class Gui(Environment):

    """This GUI environment has three locations, A, B and C. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status."""

    def __init__(self, root, height=400, width=520):
        super().__init__()
        self.status = {loc_A: 'Dirty',
                       loc_B: 'Dirty', 
                       loc_C: 'Clean',
                       loc_D: 'Clean'}
        self.root = root
        self.height = height
        self.width = width
        self.canvas = None
        self.buttons = []
        self.create_canvas()
        self.create_buttons()

    def thing_classes(self):
        """The list of things which can be used in the environment."""
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        print('Perceived Location=', agent.location,' - Perceived StatusAtLoc=',  self.status[agent.location])
        # Per farlo totalmente osservabile, bisogna ritornare le posizioni di tutte le location, con il loro status
        
        #return (agent.location, self.status[loc_A], self.status[loc_B], self.status[loc_C])
        
        return (agent.location, self.status) #(agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Change the location status (Dirty/Clean); track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        print('Executing action: ', action)
        if action == 'Right':
            if agent.location == loc_A:
                agent.location=loc_B
            else:
                agent.location=loc_C
            agent.performance -= 1
        elif action == 'Left':
            if agent.location == loc_C:
                agent.location = loc_B
            else:
                agent.location=loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                if agent.location == loc_A:
                    self.buttons[0].config(bg='white', activebackground='light grey')
                elif agent.location == loc_B:
                    self.buttons[1].config(bg='white', activebackground='light grey')
                else:
                    self.buttons[2].config(bg='white', activebackground='light grey')
                agent.performance += 10
            self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice(locs)

    ###################################################################################################
    #                        INTERFACE METHODS NOT USEFULL FOR THE AGENT/ENVIRONMENT
    ###################################################################################################
    def create_canvas(self):
        """Creates Canvas element in the GUI."""
        self.canvas = Canvas(
            self.root,
            width=self.width,
            height=self.height,
            background='powder blue')
        self.canvas.pack(side='bottom')

    def create_buttons(self):
        """Creates the buttons required in the GUI."""
        text = "ABCD"
        
        for i in range(NBUTTONS):
            x, y = pos[i]
            # Create and append the button
            # A B
            # C D
            b = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white', text=text[i])
            b.config(command=lambda btn=b: self.dirt_switch(btn))
            self.buttons.append(b)
            _ = self.canvas.create_window(x, y, anchor=N, window=b)
            
    def dirt_switch(self, button):
        """Gives user the option to put dirt in any tile."""
        bg_color = button['bg']
        if bg_color == 'saddle brown':
            button.config(bg='white', activebackground='light grey')
        elif bg_color == 'white':
            button.config(bg='saddle brown', activebackground='light goldenrod')

    def read_env(self):
        """Reads the current state of the GUI."""
        
        for i, btn in enumerate(self.buttons):
            
            if btn['bg'] == 'white':
                self.status[locs[i]] = 'Clean'
            else:
                self.status[locs[i]] = 'Dirty'
                

    def update_env(self, agent):
        """Updates the GUI according to the agent's action."""
        self.read_env()
        print(self.status)
        before_step = agent.location
        self.step()   #execute_actions
        print('Current Environment Status: ', self.status)
        print('Current Agent Location: ',agent.location)
        move_agent(self, agent, before_step)

###################################################################################################
#                                           THE AGENT
###################################################################################################

def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment. [Figure 2.8]
    >>> agent = ReflexVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """
    
    def program(percept):
        
        location, status = percept
        print('Perceived Status: ', percept)
          
        
    return Agent(program)

###################################################################################################
#                            INTERFACE METHODS TO CREATE AND MOVE THE AGENT
###################################################################################################

def create_agent(env, agent, MyImage):
    """Creates the agent in the GUI and is kept independent of the environment."""
    env.add_thing(agent)
    
    i = locs.index(agent.location)
    p = pos[i]

    x, y = p[0], p[1] - 50
    
    env.text = env.canvas.create_text(x, y, font="Helvetica 10 bold italic", text="Agent")
    env.agent_logo = env.canvas.create_image(x, y, image=MyImage)


def move_agent(env, agent, before_step):
    """Moves the agent in the GUI when 'next' button is pressed."""
    if agent.location == before_step:
        pass
    else:
        if agent.location == (2, 0) and before_step == (1,0):
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (1, 0) and  before_step == (2,0):
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)
        elif agent.location == (1, 0) and  before_step == (0,0): 
            env.canvas.move(env.text, 120, 0)
            env.canvas.move(env.agent_logo, 120, 0)
        elif agent.location == (0, 0) and before_step == (1,0):
            env.canvas.move(env.text, -120, 0)
            env.canvas.move(env.agent_logo, -120, 0)

# TODO: Add more agents to the environment.
# TODO: Expand the environment to XYEnvironment.

###################################################################################################
#                                           MAIN LOOP
###################################################################################################
def main():
    """The main function of the program."""
    root = Tk()
    root.title("Vacuum Environment")
    root.geometry("500x500")
    root.resizable(0, 0)
    frame = Frame(root, bg='black')
    
    MyImage = PhotoImage(file="AgentVacuum.png")

    next_button = Button(frame, text='Next', height=2, width=6, padx=2, pady=2)
    next_button.pack(side='left')
    frame.pack(side='bottom')
    
    env = Gui(root)
    agent = ReflexVacuumAgent()
    create_agent(env, agent, MyImage)
    
    next_button.config(command=lambda: env.update_env(agent))
    
    root.mainloop()


if __name__ == "__main__":
    main()
